## PromptTemplate
- PromptTemplate(https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html#langchain_core.prompts.prompt.PromptTemplate)

In [4]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
        あなたは優秀なアシスタントです。ユーザーからの質問に回答してください。
        質問: {question}
    """
)

# プロンプトを生成して表示
prompt_value = prompt.invoke({"question": "日本の首都はどこですか？"})
print(prompt_value) # プロンプトが生成された。実際にはこのプロンプトをLLMに送る

text='\n        あなたは優秀なアシスタントです。ユーザーからの質問に回答してください。\n        質問: 日本の首都はどこですか？\n    '


## ChatPromptTemplate
`PromptTemplate`を会話形式を前提としたチャットモデル(e.g. `Chat Completions API`)に対応させたもの

- `ChatPromptTemplate`(https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate)

### from_messages()
明確にロールを分けて会話を再現したい時
  - https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate.from_messages

リストの`tuple`に設定したKeyに応じて、`SystemMessage`や`HumanMessage`(`BaseMessage`を継承)などのインスタンスとしてメッセージが生成される。

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは優秀なアシスタントです。ユーザーからの質問に回答してください。"), # tupleとしてメッセージを定義 -> SystemMessageとして扱われる
        ("human", "{question}"),  # -> HumanMessageとして扱われる
        ("ai", "東京です。")
    ]
)

# プロンプトを生成して表示
prompt_value = prompt.invoke({"question": "日本の首都はどこですか？"})
print(prompt_value) # プロンプトが生成された。実際にはこのプロンプトをLLMに送る

messages=[SystemMessage(content='あなたは優秀なアシスタントです。ユーザーからの質問に回答してください。', additional_kwargs={}, response_metadata={}), HumanMessage(content='日本の首都はどこですか？', additional_kwargs={}, response_metadata={}), AIMessage(content='東京です。', additional_kwargs={}, response_metadata={})]


### BaseMessageを継承したクラスを使ってメッセージを定義するパターン
これも同じ意味

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="あなたは優秀なアシスタントです。ユーザーからの質問に回答してください。"),
        HumanMessage(content="{question}"),
        AIMessage(content="東京です。")
    ]
)

# プロンプトを生成して表示
prompt_value = prompt.invoke({"question": "日本の首都はどこですか？"})
print(prompt_value) # プロンプトが生成された。実際にはこのプロンプトをLLMに送る

messages=[SystemMessage(content='あなたは優秀なアシスタントです。ユーザーからの質問に回答してください。', additional_kwargs={}, response_metadata={}), HumanMessage(content='{question}', additional_kwargs={}, response_metadata={}), AIMessage(content='東京です。', additional_kwargs={}, response_metadata={})]


### from_template()
シンプルにプロンプトを生成したい時
- https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate.from_template

`from_template()`を使うと全て`HumanMessage`として扱われる。

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    system: あなたは優秀なアシスタントです。ユーザーからの質問に回答してください。
    human: {question}
    """
)

# プロンプトを生成して表示
prompt_value = prompt.invoke({"question": "日本の首都はどこですか？"})
print(prompt_value) # プロンプトが生成された。実際にはこのプロンプトをLLMに送る

messages=[HumanMessage(content='\n    system: あなたは優秀なアシスタントです。ユーザーからの質問に回答してください。\n    human: 日本の首都はどこですか？\n    ', additional_kwargs={}, response_metadata={})]


### MessagePlaceHolder

In [7]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは優秀なアシスタントです。ユーザーからの質問に回答してください。"),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

prompt_value = prompt.invoke(
    {
        "chat_history": [
            HumanMessage(content="こんにちは！私はジョンと言います。"),
            AIMessage(content="こんにちは！ジョンさん。どのようにお手伝いできますか？"),
        ],
        "input": "私の名前が分かりますか？",
    }
)

print(prompt_value) 

# for message in prompt_value.messages:
#     print(message)

messages=[SystemMessage(content='あなたは優秀なアシスタントです。ユーザーからの質問に回答してください。', additional_kwargs={}, response_metadata={}), HumanMessage(content='こんにちは！私はジョンと言います。', additional_kwargs={}, response_metadata={}), AIMessage(content='こんにちは！ジョンさん。どのようにお手伝いできますか？', additional_kwargs={}, response_metadata={}), HumanMessage(content='私の名前が分かりますか？', additional_kwargs={}, response_metadata={})]


`Placeholder`を含める場合は`messages`はtupleのリスト(=`ChatPromptTemplate`型)である必要がある

In [21]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは優秀なアシスタントです。ユーザーからの質問に回答してください。"),
        ("placeholder", "{input}"),  # placeholderを指定した場合、valueはtupleのリスト(=ChatPromptTemplate型)である必要がある
    ]
)

prompt_value = prompt.invoke({"input": [("user", "日本の首都はどこですか？"), ("ai", "東京です。")]})

print(prompt_value) 

messages=[SystemMessage(content='あなたは優秀なアシスタントです。ユーザーからの質問に回答してください。', additional_kwargs={}, response_metadata={}), HumanMessage(content='日本の首都はどこですか？', additional_kwargs={}, response_metadata={}), AIMessage(content='東京です。', additional_kwargs={}, response_metadata={})]


## PlaceHolder検証
各ケースにおけるPlaceHolderの挙動を検証

In [9]:
from langchain_core.prompts import ChatPromptTemplate


print("================= Case1: 通常のテキスト(str)にPlaceholderを使用する場合 ===================")
# Case1: 通常のテキスト内でPlaceholderを使用する場合
question = "日本の首都はどこですか？"

text_prompt = f"""
あなたは優秀なアシスタントです。ユーザーからの質問に回答してください。
質問: {question}
"""
print(text_prompt) # {question}にquestionとして定義した文字列が埋め込まれる

print("================= Case2: ChatPromptTemplateでPlaceholderを使用する場合 ===================")
# Case2: ChatPromptTemplateでPlaceholderを使用する場合
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは優秀なアシスタントです。ユーザーからの質問に回答してください。"),
        ("human", "{question}"),
    ]
)

print("================= invoke前 ===================")
print(prompt) # 定義した時点ではPlaceholderが展開されない
print(type(prompt))

print("================= invoke後 ===================")
# プロンプトを生成して表示
prompt_value = prompt.invoke({f"question": {question}})  # invokeを実行するとPlaceholderが展開される
print(prompt_value)
print(type(prompt))

================= Case1: 通常のテキスト(str)にPlaceholderを使用する場合 ===================

あなたは優秀なアシスタントです。ユーザーからの質問に回答してください。
質問: 日本の首都はどこですか？

================= Case2: ChatPromptTemplateでPlaceholderを使用する場合 ===================
================= invoke前 ===================
input_variables=['question'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='あなたは優秀なアシスタントです。ユーザーからの質問に回答してください。'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]
<class 'langchain_core.prompts.chat.ChatPromptTemplate'>
================= invoke後 ===================
messages=[SystemMessage(content='あなたは優秀なアシスタントです。ユーザーからの質問に回答してください。', additional_kwargs={}, response_metadata={}), HumanMessage(content="{'日本の首都はどこですか？'}", additional_kwargs={}, response_metadata={})]
<class 'langcha